In [1]:
## Load mhsaa data into a dataframe to get valid school ids

# mhsaa_df = pd.read_csv('..\data\MHSAA_School_Data_with_Fields.csv')

## get a list of valid school ids
# valid_school_ids = mhsaa_df['SchoolId'].tolist()

# mhsaa_df.head()







In [2]:
## Read the MHSAA JSON straight into pandas

import time
import requests
import pandas as pd
from tqdm import tqdm
import numpy as np
import json
import requests


### Global variable for test
# year = 2019

In [3]:
## Url builder function

url_a = 'https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID='
url_b = '&sportTypeCode=BB&gender=M&level=V&year='
url_c = '&userid=-1'

def url_builder(school_id, year):
    url = url_a + str(school_id) + url_b + str(year) + url_c
    return url

In [4]:
# ### Test url
# year = 2022
# url = url_builder(3811, year)


# ## Get the data from the url



# print(data)

In [5]:
### 

In [6]:
# ### Set Vartiables

# # # Start of school IDs
# school_id_start = 3800
# # # End of school IDs
# school_id_end = 4999

### TEMP FOR TESTING
# # Start of school IDs
# school_id_start = 4000
# # # End of school IDs
# school_id_end = 4050

# id_list = list(range(school_id_start, school_id_end))

## Create empty dataframe to hold data
# all_games_df = pd.DataFrame()


In [7]:
### Working Block with all Game info Included

def get_team_schedule(data):
    records = []
    team_name = data['Record']['TeamName']
    popular_name = data['Record']['PopularName']
    league_name = data['League']['Name']
    season = year

    for contest in data['Contests']:
        date = contest['DateText']
        location = contest['HomeAwayCode']
        result = contest['WinLossCode']
        league_game = contest['IsLeagueGame']
        
        game_status_type = contest['GameStatusType']
        
        season_code = contest['SeasonType']
        contest_type_code = contest['ContestType']
        
        tournament_name = contest['TournamentName']
        tournament_info = contest['TournamentInfo']
        tournament_type = contest['TournamentType']
        
        post_season_info = contest['PostSeasonInfo']



        # get opponent team info
        opponent = contest['Opponents'][0]['PopularName'] if contest['Opponents'] else ''

        # Check if "ScoreText" exists and it contains '-'
        if "ScoreText" in contest and '-' in contest["ScoreText"]:
            scores = contest["ScoreText"].split('-')  # Split the ScoreText to get the scores
            scores_notes = [(score.split(" ")[0], " ".join(score.split(" ")[1:])) if " " in score else (score, "") for score in scores]

            # Check if scores_notes has at least two items
            if len(scores_notes) >= 2:
                primary_team_score, opponent_score, primary_team_note, opponent_note = None, None, None, None

                # Check if the scores are in the correct order
                if (contest["WinLossCode"] == 'W' and int(scores_notes[0][0]) > int(scores_notes[1][0])) or \
                (contest["WinLossCode"] == 'L' and int(scores_notes[0][0]) < int(scores_notes[1][0])):
                    primary_team_score = int(scores_notes[0][0])
                    opponent_score = int(scores_notes[1][0])
                    primary_team_note = scores_notes[0][1]
                    opponent_note = scores_notes[1][1]
                else:  # If the scores are not in the correct order, switch them
                    primary_team_score = int(scores_notes[1][0])
                    opponent_score = int(scores_notes[0][0])
                    primary_team_note = scores_notes[1][1]
                    opponent_note = scores_notes[0][1]
            else:  # If scores_notes has less than two items, append NaNs
                primary_team_score, opponent_score, primary_team_note, opponent_note = np.nan, np.nan, None, None
                print(f"Warning: Unexpected score format for contest {contest}")
        else:
            primary_team_score, opponent_score, primary_team_note, opponent_note = None, None, None, None

        record = {
            'TeamName': team_name,
            'PopularName': popular_name,
            'LeagueName': league_name,
            'IsLeagueGame': league_game,
            "SeeasonYear": season,
            'Date': date,
            'Location': location,
            
            
            'Result': result,
            'PrimaryTeamScore': primary_team_score,
            'PrimaryTeamNote': primary_team_note,
            'Opponent': opponent,
            'OpponentScore': opponent_score,
            'OpponentNote': opponent_note,
            
            ## New fields
            
            'GameStatusType': game_status_type,
            'SeasonCode': season_code,
            'ContestTypeCode': contest_type_code,
            'TournamentName': tournament_name,
            'TournamentInfo': tournament_info,
            'TournamentType': tournament_type,
            'PostSeasonInfo': post_season_info

        }

        records.append(record)
    return pd.DataFrame(records)



In [8]:
# ## send a single request to get the team info

# school_id = 3811
# year = 2021

# # Send a request to get the team info

# page = url_builder(school_id)

# response = requests.get(page)
# data = response.json()

# team_info = data.get("Contests", {})
# # team_info


# ## send a single request to get the team schedule


# get_team_schedule(data, data["Record"])

# # all_games_df.head()


In [9]:
### Set Up a loop to parse a block of school IDs and years

from tqdm import tqdm
import time

# year
year_range = range(2001, 2023)

# Start of school IDs
school_id_start = 3000
# End of school IDs
school_id_end = 4999

# Example of a single school ID
ex_url = 'https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID=3835&sportTypeCode=BB&gender=M&level=V&year=2021&userid=-1'

# Create empty dataframe to hold data
all_games_df = pd.DataFrame()

# Create a list of school IDs
id_list = list(range(school_id_start, school_id_end))

# total number of iterations for the progress bar
total = len(year_range) * len(id_list)

with tqdm(total=total) as pbar:
    for year in year_range:
        # Loop through the list of school IDs
        for school_id in id_list:
            try:
                url = url_builder(school_id, year)
                response = requests.get(url)
                data = response.json()
                all_games_df = all_games_df.append(get_team_schedule(data), ignore_index=True)
            except Exception as e:
                print(f"Failed to get data for school_id: {school_id}, year: {year}. Error: {e}")
            finally:
                pbar.update()  # update the progress bar after each iteration
                time.sleep(0.1)  # optional delay to avoid hammering the server

all_games_df.head()



  0%|          | 0/43978 [00:00<?, ?it/s]C:\Users\Justin\AppData\Local\Temp\ipykernel_13552\3238233421.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_games_df = all_games_df.append(get_team_schedule(data), ignore_index=True)
 16%|█▌        | 6890/43978 [47:56<4:39:17,  2.21it/s] C:\Users\Justin\AppData\Local\Temp\ipykernel_13552\3238233421.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_games_df = all_games_df.append(get_team_schedule(data), ignore_index=True)
 16%|█▌        | 6994/43978 [48:42<4:17:29,  2.39it/s]C:\Users\Justin\AppData\Local\Temp\ipykernel_13552\3238233421.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_games_df = all_games_df.append(get_team_schedule(data), ignore_index=True)
 

Failed to get data for school_id: 3602, year: 2006. Error: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


 38%|███▊      | 16826/43978 [1:55:55<2:41:35,  2.80it/s] C:\Users\Justin\AppData\Local\Temp\ipykernel_13552\3238233421.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_games_df = all_games_df.append(get_team_schedule(data), ignore_index=True)
 38%|███▊      | 16827/43978 [1:55:58<7:42:44,  1.02s/it]C:\Users\Justin\AppData\Local\Temp\ipykernel_13552\3238233421.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_games_df = all_games_df.append(get_team_schedule(data), ignore_index=True)
 38%|███▊      | 16828/43978 [1:55:59<7:11:54,  1.05it/s]C:\Users\Justin\AppData\Local\Temp\ipykernel_13552\3238233421.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_games_df = all_games_df.append(get_team_schedule(data)

Failed to get data for school_id: 3835, year: 2013. Error: argument of type 'NoneType' is not iterable


 56%|█████▋    | 24825/43978 [2:58:32<3:08:17,  1.70it/s]C:\Users\Justin\AppData\Local\Temp\ipykernel_13552\3238233421.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_games_df = all_games_df.append(get_team_schedule(data), ignore_index=True)
 56%|█████▋    | 24826/43978 [2:58:33<4:10:57,  1.27it/s]C:\Users\Justin\AppData\Local\Temp\ipykernel_13552\3238233421.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_games_df = all_games_df.append(get_team_schedule(data), ignore_index=True)
 56%|█████▋    | 24827/43978 [2:58:34<4:53:39,  1.09it/s]C:\Users\Justin\AppData\Local\Temp\ipykernel_13552\3238233421.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_games_df = all_games_df.append(get_team_schedule(data),

...

Failed to get data for school_id: 4520, year: 2020. Error: argument of type 'NoneType' is not iterable


C:\Users\Justin\AppData\Local\Temp\ipykernel_13552\3238233421.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_games_df = all_games_df.append(get_team_schedule(data), ignore_index=True)
 90%|████████▉ | 39503/43978 [5:02:09<41:32,  1.80it/s]C:\Users\Justin\AppData\Local\Temp\ipykernel_13552\3238233421.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_games_df = all_games_df.append(get_team_schedule(data), ignore_index=True)
 90%|████████▉ | 39504/43978 [5:02:10<41:59,  1.78it/s]C:\Users\Justin\AppData\Local\Temp\ipykernel_13552\3238233421.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_games_df = all_games_df.append(get_team_schedule(data), ignore_index=True)
 90%|████████▉ | 39505/43978 [5:02:10<41:

Failed to get data for school_id: 4642, year: 2021. Error: [Errno Expecting value] <!DOCTYPE html>
<html>
    <head>
        <title>Runtime Error</title>
        <meta name="viewport" content="width=device-width" />
        <style>
         body {font-family:"Verdana";font-weight:normal;font-size: .7em;color:black;} 
         p {font-family:"Verdana";font-weight:normal;color:black;margin-top: -5px}
         b {font-family:"Verdana";font-weight:bold;color:black;margin-top: -5px}
         H1 { font-family:"Verdana";font-weight:normal;font-size:18pt;color:red }
         H2 { font-family:"Verdana";font-weight:normal;font-size:14pt;color:maroon }
         pre {font-family:"Consolas","Lucida Console",Monospace;font-size:11pt;margin:0;padding:0.5em;line-height:14pt}
         .marker {font-weight: bold; color: black;text-decoration: none;}
         .version {color: gray;}
         .error {margin-bottom: 10px;}
         .expandable { text-decoration:underline; font-weight:bold; color:navy; curs

 95%|█████████▍| 41624/43978 [5:20:55<17:56,  2.19it/s]C:\Users\Justin\AppData\Local\Temp\ipykernel_13552\3238233421.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_games_df = all_games_df.append(get_team_schedule(data), ignore_index=True)
 95%|█████████▍| 41628/43978 [5:20:57<19:07,  2.05it/s]C:\Users\Justin\AppData\Local\Temp\ipykernel_13552\3238233421.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_games_df = all_games_df.append(get_team_schedule(data), ignore_index=True)
 95%|█████████▍| 41630/43978 [5:20:58<23:14,  1.68it/s]C:\Users\Justin\AppData\Local\Temp\ipykernel_13552\3238233421.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_games_df = all_games_df.append(get_team_schedule(data), ignor

,TeamName,PopularName,LeagueName,IsLeagueGame,Date,Location,Result,PrimaryTeamScore,PrimaryTeamNote,Opponent,OpponentScore,OpponentNote,GameStatusType,SeasonCode,ContestTypeCode,TournamentName,TournamentInfo,TournamentType,PostSeasonInfo
0,Detroit Mumford,Detroit Mumford,,True,12/16/2004,A,,None,None,Mt Clemens,None,None,NOR,1,1,None,,0,
1,Mt Clemens,Mount Clemens,,True,12/16/2004,H,,None,None,Detroit Mumford,None,None,NOR,1,1,None,,0,
2,Brighton,Brighton,Kensington Lakes Activities Association,False,12/9/2009,A,W,47,,Grosse Pointe South,46,,NOR,0,1,None,,None,
3,Brighton,Brighton,Kensington Lakes Activities Association,False,12/11/2009,H,W,55,,Ypsilanti Lincoln,39,,NOR,0,1,None,,None,
4,Brighton,Brighton,Kensington Lakes Activities Association,False,12/15/2009,H,W,61,,Fowlerville,54,,NOR,0,1,None,,None,


In [10]:
all_games_df.tail(25)

,TeamName,PopularName,LeagueName,IsLeagueGame,Date,Location,Result,PrimaryTeamScore,PrimaryTeamNote,Opponent,OpponentScore,OpponentNote,GameStatusType,SeasonCode,ContestTypeCode,TournamentName,TournamentInfo,TournamentType,PostSeasonInfo
202884,South Lyon East,South Lyon East,Lakes Valley Conference,False,12/9/2022,H,W,76.0,,Pinckney,47.0,,NOR,1,1,None,,0,
202885,South Lyon East,South Lyon East,Lakes Valley Conference,False,12/13/2022,A,W,74.0,,Livonia Churchill,35.0,,NOR,1,1,None,,0,
202886,South Lyon East,South Lyon East,Lakes Valley Conference,True,12/16/2022,H,W,75.0,,Walled Lake Northern,55.0,,NOR,1,1,None,,0,
202887,South Lyon East,South Lyon East,Lakes Valley Conference,True,12/19/2022,H,L,58.0,,Walled Lake Western,73.0,,NOR,1,1,None,,0,
202888,South Lyon East,South Lyon East,Lakes Valley Conference,False,12/21/2022,A,L,73.0,,Salem,85.0,,NOR,1,1,None,,0,
202889,South Lyon East,South Lyon East,Lakes Valley Conference,False,12/28/2022,H,W,72.0,,Auburn Hills Avondale,60.0,,NOR,1,1,None,,0,
202890,South Lyon East,South Lyon East,Lakes Valley Conference,True,1/10/2023,H,W,64.0,,White Lake Lakeland,60.0,,NOR,1,1,None,,0,
202891,South Lyon East,South Lyon East,Lakes Valley Conference,True,1/13/2023,H,W,66.0,,South Lyon,58.0,,NOR,1,1,None,,0,
202892,South Lyon East,South Lyon East,Lakes Valley Conference,False,1/15/2023,A,W,73.0,,Troy Athens,51.0,,NOR,1,1,None,,0,
202893,South Lyon East,South Lyon East,Lakes Valley Conference,True,1/17/2023,A,W,52.0,,Walled Lake Central,46.0,,NOR,1,1,None,,0,


In [11]:
### Save it as a CSV to check

all_games_df.to_csv('MBB_SCRAPE_PARSE_v2.csv', index=False)

In [ ]:
### ADDED THIS ON THE SECOND RUN with more years - all the way down to 2000

### Set Up a loop to parse a block of school IDs and years

from tqdm import tqdm
import time

# year
year_range = range(2000, 2011)

# Start of school IDs
school_id_start = 3000
# End of school IDs
school_id_end = 4999

# Example of a single school ID
ex_url = 'https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID=3835&sportTypeCode=BB&gender=M&level=V&year=2021&userid=-1'

# Create empty dataframe to hold data
all_games_df = pd.DataFrame()

# Create a list of school IDs
id_list = list(range(school_id_start, school_id_end))

# total number of iterations for the progress bar
total = len(year_range) * len(id_list)

with tqdm(total=total) as pbar:
    for year in year_range:
        # Loop through the list of school IDs
        for school_id in id_list:
            try:
                url = url_builder(school_id, year)
                response = requests.get(url)
                data = response.json()
                all_games_df = all_games_df.append(get_team_schedule(data), ignore_index=True)
            except Exception as e:
                print(f"Failed to get data for school_id: {school_id}, year: {year}. Error: {e}")
            finally:
                pbar.update()  # update the progress bar after each iteration
                time.sleep(0.1)  # optional delay to avoid hammering the server

all_games_df.head()

In [ ]:
### Save it as a CSV to check

all_games_df.to_csv('SCRAPE_PARSE_v1_extended.csv', index=False)

In [ ]:
endasergsdfhsdf

In [ ]:
# ## test the example url

# response = requests.get(ex_url)
# data = response.json()
# games_df = games_df.append(get_team_schedule(data, data["Record"]), ignore_index=True)

In [ ]:
eneneneene

In [ ]:
import time
import requests
import pandas as pd
from tqdm import tqdm

# Your team schedule function
def get_team_schedule(data, team_info):
    
    # Initialize lists to store the game information
    date_time = []
    primary_team_name = []
    primary_team_id = []
    game_result = []
    opponent_name = []
    opponent_id = []
    location = []
    game_type = []
    tournament_name = []
    tournament_info = []
    is_league_game = []
    primary_team_score = []
    opponent_score = []
    primary_team_note = []
    opponent_note = []

    # Unpack the game information
    for contest in data.get("Contests", []):  # Default to an empty list if "Contests" is not in the JSON response      
        date_time.append(contest["StartDate"])
        primary_team_name.append(team_info["TeamName"])  # Primary team name is the same for all games
        primary_team_id.append(contest["SportTeamId"])  
        game_result.append(contest["WinLossCode"])
        opponent_name.append(contest["Opponents"][0]["PopularName"])
        opponent_id.append(contest["Opponents"][0]["SportTeamId"])
        location.append(contest["HomeAwayCode"])
        game_type.append(contest["SeasonType"])
        tournament_name.append(contest["TournamentName"])
        tournament_info.append(contest["TournamentInfo"])
        is_league_game.append(contest["IsLeagueGame"])

        scores = contest["ScoreText"].split('-')  # Split the ScoreText to get the scores
        scores_notes = [(score.split(" ")[0], " ".join(score.split(" ")[1:])) if " " in score else (score, "") for score in scores]

         # Check if "ScoreText" exists and it contains '-'
        if "ScoreText" in contest and '-' in contest["ScoreText"]:
            scores = contest["ScoreText"].split('-')  # Split the ScoreText to get the scores
            scores_notes = [(score.split(" ")[0], " ".join(score.split(" ")[1:])) if " " in score else (score, "") for score in scores]

            # Check if scores_notes has at least two items
            if len(scores_notes) >= 2:

                # # Check if the scores are in the correct order
                if (contest["WinLossCode"] == 'W' and int(scores_notes[0][0]) > int(scores_notes[1][0])) or \
                (contest["WinLossCode"] == 'L' and int(scores_notes[0][0]) < int(scores_notes[1][0])):
                    primary_team_score.append(int(scores_notes[0][0]))
                    opponent_score.append(int(scores_notes[1][0]))
                    primary_team_note.append(scores_notes[0][1])
                    opponent_note.append(scores_notes[1][1])
                else:  # If the scores are not in the correct order, switch them
                    primary_team_score.append(int(scores_notes[1][0]))
                    opponent_score.append(int(scores_notes[0][0]))
                    primary_team_note.append(scores_notes[1][1])
                    opponent_note.append(scores_notes[0][1])
            else:  # If scores_notes has less than two items, append NaNs
                primary_team_score.append(np.nan)
                print(f"Warning: Unexpected score format for contest {contest}")

    # Create the dataframe
    games_df = pd.DataFrame({
        "DateTime": date_time,
        "PrimaryTeamName": primary_team_name,
        "PrimaryTeamID": primary_team_id,
        "PrimaryTeamScore": primary_team_score,
        "PrimaryTeamNote": primary_team_note,
        "Result": game_result,
        "OpponentName": opponent_name,
        "OpponentID": opponent_id,
        "OpponentScore": opponent_score,
        "OpponentNote": opponent_note,
        "Location": location,
        "GameType": game_type,
        "TournamentName": tournament_name,
        "TournamentInfo": tournament_info,
        "IsLeagueGame": is_league_game,
    })
    
    return games_df




In [ ]:




# ### Set Vartiables
## Url builder function
urls = []
url_a = 'https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID='
url_b = '&sportTypeCode=BB&gender=M&level=V&year='
url_c = '&userid=-1'

###### Paramenters 
school_ids = id_list
years = range(2011, 2022)

def url_builder(school_id, year):  # Make sure to pass 'year' as a parameter
    url = url_a + str(school_id) + url_b + str(year) + url_c
    return url  # you only need to return the url, no need to append to a list here





# Loop through each url, retrieve data, process it and append it to the all_games_df dataframe
for url in tqdm(urls):
    r = requests.get(url)
    if r.status_code == 200:  # Check if the request was successful
        data = r.json()

        # Check if "TeamInfo" is in the data
        if "TeamInfo" in data:
            # Extract team_info from the data
            team_info = data["TeamName"]
    
            # Call the get_team_schedule function and append the result to all_games_df
            all_games_df = all_games_df.append(get_team_schedule(data, team_info), ignore_index=True)
        else:
            # print(f"Warning: 'TeamInfo' not found in data for URL {url}")
            pass
    

    # Add a short delay to avoid too many requests too quickly
    time.sleep(1)  # sleep for 1 second


In [ ]:
## Look at the data
all_games_df.info()

In [ ]:
stoppp

In [ ]:
## Link address for a single school's boy's basketball data

url_ex = 'https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID=4427&sportTypeCode=BB&gender=M&level=V&year=2022&userid=-1'

# Download the JSON
r = requests.get(url_ex)
# Convert to a dictionary
data = r.json()

# Print the dictionary
# print(data)

## pretty print the dictionary
# print(json.dumps(data, indent=4, sort_keys=True))

In [ ]:

## Url builder function

url_a = 'https://my.mhsaa.com/DesktopModules/MHSAA-Async-SportTeamSchedule/Endpoint.ashx?&method=schedules&orgID='
url_b = '&sportTypeCode=BB&gender=M&level=V&year='
url_c = '&userid=-1'

def url_builder(school_id):
    url = url_a + str(school_id) + url_b + str(year) + url_c
    return url

## I want to take a list of school ids and a range of years and return a dataframe of all the data

# Set the year range - the year reflects the start academic year
year_start = 2010
year_end = 2022

# Create a list of years
years = list(range(year_start, year_end))

# Create a list of school ids
school_ids = valid_school_ids

# Create a list of urls
urls = []

for school_id in tqdm(school_ids):
    for year in years:
        url = url_builder(school_id)
        urls.append(url)


# Create a list of dataframes
dfs = []

for url in tqdm(urls):
    r = requests.get(url)
    data = r.json()
    df = pd.DataFrame(data)
    dfs.append(df)

# Concatenate the dataframes
master_mhsaa_basketball_df = pd.concat(dfs)

# Save the dataframe
master_mhsaa_basketball_df.to_csv('..\TEMP\master_mhsaa_basketball_df.csv', index=False)



    




In [ ]:
## nOT GOING TO DO IT THIS WAY. THERE ARE NESTED RECORD IN THE JSON FOR STANDING FOR THE WHOLE CONFERENCE
## IT'S UNESS TO SCRAPE CAN GET ANOTHER WAY

# ################### TEAM SUMMARY #####################
# ## Create a dataframe with the team summary information

# # Initialize lists to store the team information
# team_name = []
# popular_name = []
# school_sport_team_id = []
# games_behind = []
# conference_wins = []
# conference_losses = []
# conference_ties = []
# overall_wins = []
# overall_losses = []
# overall_ties = []
# total_games_played = []
# school_short_name = []



# # Unpack the team information
# for contest in data["Contests"]:
#     team_name.append(team_info["TeamName"])
#     popular_name.append(team_info["PopularName"])
#     school_sport_team_id.append(team_info["SchoolSportTeamId"])
#     games_behind.append(team_info["GamesBehind"])
#     conference_wins.append(team_info["ConferenceNumberOfWins"])
#     conference_losses.append(team_info["ConferenceNumberOfLosses"])
#     conference_ties.append(team_info["ConferenceNumberOfTies"])
#     overall_wins.append(team_info["OverallNumberOfWins"])
#     overall_losses.append(team_info["OverallNumberOfLosses"])
#     overall_ties.append(team_info["OverallNumberOfTies"])
#     total_games_played.append(team_info["TotalNumberOfGamesPlayed"])
#     school_short_name.append(team_info["SchoolShortName"])

# # Create the dataframe
# team_summary_df = pd.DataFrame({
#     "TeamName": team_name,
#     "PopularName": popular_name,
#     "SchoolSportTeamId": school_sport_team_id,
#     "GamesBehind": games_behind,
#     "ConferenceWins": conference_wins,
#     "ConferenceLosses": conference_losses,
#     "ConferenceTies": conference_ties,
#     "OverallWins": overall_wins,
#     "OverallLosses": overall_losses,
#     "OverallTies": overall_ties,
#     "TotalGamesPlayed": total_games_played,
#     "SchoolShortName": school_short_name
# })

# # }])

In [ ]:


# Resulting game level dataframe
games_df.tail(20)